<a href="https://colab.research.google.com/github/bs21b038/assignment2data/blob/main/da6401_a3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Transliteration system

import pandas as pd
import numpy as np

train_df= pd.read_csv('te.translit.sampled.train.tsv',sep='\t',header=None)
train_df.columns = ['devanagari','latin','serial']
train_df

,devanagari,latin,serial
0,అంకిత,amkita,1
1,అంకిత,ankita,2
2,అంకిత,ankitha,2
3,అంకితం,ankitam,1
4,అంకితం,ankitham,2
...,...,...,...
58545,హ్యుమానిటీస్,huumanities,1
58546,హ్యూమన్,human,3
58547,హ్యూమన్,humane,1
58548,హ్యూమన్,huumane,1


In [47]:
seq=train_df['latin'][0]
seq

'amkita'

In [48]:
list(seq)
v=set()
print(v)
v.update(list(seq))
sorted(list(v))

set()


['a', 'i', 'k', 'm', 't']

In [49]:
#Vocab dictionaries
train_df=train_df.dropna()

def build_vocab(sequences):
    vocab = set()
    for seq in sequences:
        vocab.update(list(seq))
    vocab = sorted(list(vocab))
    vocab = ['<pad>', '<sos>', '<eos>'] + vocab  # special tokens
    char2idx = {ch: idx for idx, ch in enumerate(vocab)}
    idx2char = {idx: ch for ch, idx in char2idx.items()}
    return char2idx, idx2char

input_char2idx, input_idx2char = build_vocab(train_df["latin"])
target_char2idx, target_idx2char = build_vocab(train_df["devanagari"])

print("Input vocab size:", len(input_char2idx))
print("Target vocab size:", len(target_char2idx))

Input vocab size: 29
Target vocab size: 66


In [50]:
input_char2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28}

In [51]:
target_char2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 'ం': 3,
 'ః': 4,
 'అ': 5,
 'ఆ': 6,
 'ఇ': 7,
 'ఈ': 8,
 'ఉ': 9,
 'ఊ': 10,
 'ఋ': 11,
 'ఎ': 12,
 'ఏ': 13,
 'ఐ': 14,
 'ఒ': 15,
 'ఓ': 16,
 'ఔ': 17,
 'క': 18,
 'ఖ': 19,
 'గ': 20,
 'ఘ': 21,
 'చ': 22,
 'ఛ': 23,
 'జ': 24,
 'ఝ': 25,
 'ఞ': 26,
 'ట': 27,
 'ఠ': 28,
 'డ': 29,
 'ఢ': 30,
 'ణ': 31,
 'త': 32,
 'థ': 33,
 'ద': 34,
 'ధ': 35,
 'న': 36,
 'ప': 37,
 'ఫ': 38,
 'బ': 39,
 'భ': 40,
 'మ': 41,
 'య': 42,
 'ర': 43,
 'ఱ': 44,
 'ల': 45,
 'ళ': 46,
 'వ': 47,
 'శ': 48,
 'ష': 49,
 'స': 50,
 'హ': 51,
 'ా': 52,
 'ి': 53,
 'ీ': 54,
 'ు': 55,
 'ూ': 56,
 'ృ': 57,
 'ె': 58,
 'ే': 59,
 'ై': 60,
 'ొ': 61,
 'ో': 62,
 'ౌ': 63,
 '్': 64,
 '\u200c': 65}

In [54]:
# vectorial representations of letters

def encode_sentence(sentence, char2idx, max_len, is_target=False):
    tokens = [char2idx[c] for c in sentence]
    if is_target:
        tokens = [char2idx["<sos>"]] + tokens + [char2idx["<eos>"]]
    return tokens + [char2idx["<pad>"]] * (max_len - len(tokens))

# Compute max lengths
max_encoder_seq_length = max(len(x) for x in train_df["latin"])
max_decoder_seq_length = max(len(y) for y in train_df["devanagari"]) + 2  # for <sos> and <eos>

print(max_encoder_seq_length)
print(max_decoder_seq_length)

25
22


In [ ]:
Embedding lookup

Encoder RNN forward steps

Decoder RNN steps

Output projection and softmax



In [53]:
for i in range(10):
  print(len(train_df['latin'][i]))

6
6
7
7
8
12
13
15
14
15
